In [1]:
import pandas as pd
from sklearn import preprocessing
import tflearn,numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import torch

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
main=pd.read_pickle("main_four.pickle")
columns=["a"," b"," c"," d"," e"," f"," g"," h"," i"," j"," k"," l"," m"," n"," o"," p"," q"," r"," s"," t"," u"," v"," w"," x"," y"," z","z1"]

new_splited_only=pd.DataFrame(data=main.x.tolist(),columns=columns)

In [ ]:
to_bar=[]#[columns[0],columns[4],columns[17],columns[24],columns[6],columns[7],columns[8],columns[10],columns[11],columns[13],columns[14]]#" e"," g"," r"," u"," v"," y"]
emps=[]
for i in columns:
    if i in to_bar:
        pass
    else:
        scaler=preprocessing.minmax_scale(X=new_splited_only["%s"%i],feature_range=(0.2,1),)
        #processed_column=scaler.transform(new_splited_only["%s"%i])
        emps.append(scaler)
    
scaled_frame=pd.DataFrame()
for i in range(len(columns)-len(to_bar)):
        scaled_frame["%s"%columns[i]]=emps[i]
    
Y3=main.y.tolist()
y_p=savgol_filter(Y3,11,3) 

In [6]:
Y=main.y.tolist()
X=scaled_frame.values.tolist()
X_test=X[52000:52500]

Y_test=Y[52000:52500]
Y_test=preprocessing.minmax_scale(Y_test,feature_range=(0.2,1)).tolist()
Y_test=[[y] for y  in Y_test]
X=X[:52000]
Y=Y[:52000]
Y=[[x] for x in y_p]
y_processed=preprocessing.minmax_scale(Y,feature_range=(0.2,1))
y=y_processed[:52000]
Y_test=y_processed[52000:52500]
x=torch.tensor(X)
y=torch.tensor(y,dtype=torch.float64)
X_test=torch.tensor(X_test)
Y_test=torch.tensor(Y_test)

In [7]:
class net(torch.nn.Module):
    def __init__(self,D_in,H_nodes,D_out):
        super(net,self).__init__()
    
        
        
        self.tol=torch.nn.Linear(D_in,27,bias=False)
        self.tol.weight.data.fill_(.01)
        
        #self.linear1=torch.nn.Sequential(self.tol,torch.nn.Tanh())#,torch.nn.Dropout(0.2))
        
        #self.linear1=torch.nn.init.uniform_(self.linear1,0.1)
        #self.dropout=torch.nn.Dropout()
        
        self.linear2=torch.nn.Linear(27,13,bias=False)
        self.linear2.weight.data.fill_(0.01)
        self.linear3=torch.nn.Linear(13,7,bias=False)
        self.linear3.weight.data.fill_(0.01)
        self.linear4=torch.nn.Linear(7,3,bias=False)
        self.linear4.weight.data.fill_(0.01)
        self.out=torch.nn.Linear(1,1,bias=False)
        self.out.weight.data.fill_(1.0)
        
    def forward(self,x):
        
        indata=self.tol(x)
        indata=torch.tanh(indata)
        #indata1=torch.nn.Tanh(indata)
        #indata1=self.dropout()
        indata2=self.linear2(indata)
        indata2=torch.tanh(indata2)
        #indata2=torch.nn.Tanh(indata2)
        
        indata3=self.linear3(indata2)
        indata3=torch.tanh(indata3)
        #indata3=torch.nn.Tanh(indata3)
        indata4=self.linear4(indata3)
        indata4=torch.tanh(indata4)
        #indata4=torch.nn.Tanh(indata4)
        
        #indataf=self.out(indata)
        #indataf=torch.nn.Tanh(indataf)
        
        return indata
    
model=net(27,27,1)    
losss=torch.nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters())       

In [8]:
train_loss=[]
test_loss=[]
for i in range(300):
    y_pred=model(x)
    loss=losss(y_pred,y.float())
    train_loss.append(loss.item())
    #print("training loss:::",loss.item())
    new_loss=losss(model(X_test),Y_test.float())
    test_loss.append(new_loss.item())
    #print("::: test loss:::",new_loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [9]:
pre=model(x)
plt.plot(pre.detach().numpy())
plt.plot(y_processed)